In [ ]:
import numpy as np
import matplotlib.pylab as plt
from scipy.ndimage import rotate

from utils import load_cifar10, batch_plot, seed_everything

seed_everything(21)

In [ ]:
(X_train, y_train), (X_test, y_test) = load_cifar10("../code/cs231n/datasets/cifar-10-batches-py/")

assert X_train.shape == (50000, 32, 32, 3)
assert y_train.shape == (50000,)
assert X_test.shape == (10000, 32, 32, 3)
assert y_test.shape == (10000,)

In [ ]:
# Get the indexes of 'batch_size' random digits
batch_size = 16
random_indexes = np.random.randint(X_train.shape[0], size=batch_size)
# Plot digits with labels
batch_plot(X_train[random_indexes], y_train[random_indexes])

In [ ]:
def compute_image_l2_distance_broadcasting(x, y):
    """
    Compute the L2 distance between each pixel of x and each pixel of y using direct numpy broadcasting.

    Parameters
    ----------
    x : numpy.ndarray
        Array of shape (m, h, w, c) containing m images of shape (h, w, c).
    y : numpy.ndarray
        Array of shape (n, h, w, c) containing n images of shape (h, w, c).

    Returns
    -------
    numpy.ndarray
        Array of shape (m, n) containing the L2 distance between each image of x and each image of y.
    """
    # this operation will consume a lot of memory
    # (m, 1, h, w, c) - (n, h, w, c) -> (m, n, h, w, c)
    return np.sqrt(np.sum(np.square(np.expand_dims(x, axis=1) - y), axis=(2, 3, 4)))


def compute_image_l2_distance_expanded(x, y):
    """
    Compute the L2 distance between each pixel of x and each pixel of y using the expanded form of the equation.

    Parameters
    ----------
    x : numpy.ndarray
        Array of shape (m, h, w, c) containing m images of shape (h, w, c).
    y : numpy.ndarray
        Array of shape (n, h, w, c) containing n images of shape (h, w, c).

    Returns
    -------
    numpy.ndarray
        Array of shape (m, n) containing the L2 distance between each image of x and each image of y.
    """
    # (x - y)^2 = x^2 + y^2 - 2xy
    # (m, 1) + (n,) - 2(m, n) -> (m, n)
    return np.sqrt(
        np.expand_dims(np.sum(np.square(x), axis=(1, 2, 3)), axis=1)
        + np.sum(np.square(y), axis=(1, 2, 3))
        - 2 * np.einsum("mhwc,nhwc->mn", x, y)
    )


def compute_image_l2_distance_flatten(x, y):
    """
    Compute the L2 distance between each pixel of x and each pixel of y using the flattened form of the images.

    Parameters
    ----------
    x : numpy.ndarray
        Array of shape (m, h x w x c) containing m images of flattened shape (h x w x c).
    y : numpy.ndarray
        Array of shape (n, h x w x c) containing n images of flattened shape (h x w x c).

    Returns
    -------
    numpy.ndarray
        Array of shape (m, n) containing the L2 distance between each image of x and each image of y.
    """
    # (x - y)^2 = x^2 + y^2 - 2xy
    # (m, 1) + (n,) - 2(m, n) -> (m, n)
    return np.sqrt(np.sum(np.square(x), axis=1, keepdims=True) + np.sum(np.square(y), axis=1) - 2 * x @ y.T)

In [ ]:
train_size, test_size = 100, 50

X_train_batch, X_test_batch = X_train[:train_size].astype(int), X_test[:test_size].astype(int)
X_train_batch_reshaped, X_test_batch_reshaped = X_train_batch.reshape((train_size, -1)), X_test_batch.reshape(
    (test_size, -1)
)

assert np.allclose(
    compute_image_l2_distance_expanded(X_test_batch, X_train_batch),
    compute_image_l2_distance_broadcasting(X_test_batch, X_train_batch),
)

assert np.allclose(
    compute_image_l2_distance_expanded(X_test_batch, X_train_batch),
    compute_image_l2_distance_flatten(X_test_batch_reshaped, X_train_batch_reshaped),
)

%timeit compute_image_l2_distance_broadcasting(X_test_batch, X_train_batch)
%timeit compute_image_l2_distance_expanded(X_test_batch, X_train_batch)
%timeit compute_image_l2_distance_flatten(X_test_batch_reshaped, X_train_batch_reshaped)

In [ ]:
top_num = 8

train_size, test_size = 1000, 500

X_train_batch, X_test_batch = X_train[:train_size].astype(int), X_test[:test_size].astype(int)
X_train_batch_reshaped, X_test_batch_reshaped = X_train_batch.reshape((train_size, -1)), X_test_batch.reshape(
    (test_size, -1)
)

distances = compute_image_l2_distance_flatten(X_test_batch_reshaped, X_train_batch_reshaped)

test_vs_trains_sum = distances.sum(axis=1)
train_vs_tests_sum = distances.sum(axis=0)

top_test_vs_trains_differences = np.argsort(test_vs_trains_sum)[-top_num:]
top_train_vs_tests_differences = np.argsort(train_vs_tests_sum)[-top_num:]

plt.figure(figsize=(16, 8))
plt.imshow(distances, interpolation="none", cmap="gray")
plt.plot(top_train_vs_tests_differences, np.zeros_like(top_train_vs_tests_differences), "v", color="w", markersize=8)
plt.plot(np.zeros_like(top_test_vs_trains_differences), top_test_vs_trains_differences, ">", color="w", markersize=8)
plt.axis("off")
plt.show()

In [ ]:
def compute_image_l1_distance_one_loop(x, y):
    return np.array([np.sum(np.abs(i - y), axis=(1, 2, 3)) for i in x])


def compute_image_l1_distance_no_loop(x, y):
    # will consume more memory and much slower
    return np.sum(np.abs(np.expand_dims(x, axis=1) - y), axis=(2, 3, 4))


samples_train, samples_test = X_train_batch[:100], X_test_batch[:50]

assert np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_no_loop(samples_test, samples_train),
)

%timeit compute_image_l1_distance_origional_one_loop(samples_test, samples_train)
%timeit compute_image_l1_distance_origional_no_loop(samples_test, samples_train)

In [ ]:
top_num = 8

distances = compute_image_l1_distance_one_loop(X_test_batch, X_train_batch)

test_vs_trains_sum = distances.sum(axis=1)
train_vs_tests_sum = distances.sum(axis=0)

top_test_vs_trains_differences = np.argsort(test_vs_trains_sum)[-top_num:]
top_train_vs_tests_differences = np.argsort(train_vs_tests_sum)[-top_num:]

plt.figure(figsize=(16, 8))
plt.imshow(distances, interpolation="none", cmap="gray")
plt.plot(top_train_vs_tests_differences, np.zeros_like(top_train_vs_tests_differences), "v", color="w", markersize=8)
plt.plot(np.zeros_like(top_test_vs_trains_differences), top_test_vs_trains_differences, ">", color="w", markersize=8)
plt.axis("off")
plt.show()

Which of the following preprocessing steps will not change the performance of a Nearest Neighbor classifier that uses L1 distance? Select all that apply.
1. Subtracting the mean $\mu$ ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu$.)
2. Subtracting the per pixel mean $\mu_{ij}$  ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu_{ij}$.)
3. Subtracting the mean $\mu$ and dividing by the standard deviation $\sigma$.
4. Subtracting the pixel-wise mean $\mu_{ij}$ and dividing by the pixel-wise standard deviation $\sigma_{ij}$.
5. Rotating the coordinate axes of the data.

In [ ]:
samples_all = np.concatenate([samples_train, samples_test])
mu = samples_all.mean()
std = samples_all.std()
mu_ij = samples_all.mean(axis=0)
std_ij = samples_all.std(axis=0)

# 1.
assert np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_one_loop(samples_test - mu, samples_train - mu),
)

$$
|
\tilde{\mathbf{a}} - \tilde{\mathbf{b}}
|
=
\left|
\left(
\begin{bmatrix}
  a_{00} & a_{01} & \cdots & a_{0n} \\
  a_{10} & a_{11} & \cdots & a_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  a_{m0} & a_{m1} & \cdots & a_{mn} \\
\end{bmatrix} 
- \mu
\right)
-
\left(
\begin{bmatrix}
  b_{00} & b_{01} & \cdots & b_{0n} \\
  b_{10} & b_{11} & \cdots & b_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  b_{m0} & b_{m1} & \cdots & b_{mn} \\
\end{bmatrix}
- \mu
\right)
\right|
= |\mathbf{a} - \mathbf{b}| \tag{1}
$$

In [ ]:
# 2.
assert np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_one_loop(samples_test - mu_ij, samples_train - mu_ij),
)

$$
|\tilde{\mathbf{a}} - \tilde{\mathbf{b}}|
=
\left|
\left(
\begin{bmatrix}
  a_{00} & a_{01} & \cdots & a_{0n} \\
  a_{10} & a_{11} & \cdots & a_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  a_{m0} & a_{m1} & \cdots & a_{mn} \\
\end{bmatrix} 
-
\begin{bmatrix}
  \mu_{00} & \mu_{01} & \cdots & \mu_{0n} \\
  \mu_{10} & \mu_{11} & \cdots & \mu_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  \mu_{m0} & \mu_{m1} & \cdots & \mu_{mn} \\
\end{bmatrix} 
\right)
-
\left(
\begin{bmatrix}
  b_{00} & b_{01} & \cdots & b_{0n} \\
  b_{10} & b_{11} & \cdots & b_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  b_{m0} & b_{m1} & \cdots & b_{mn} \\
\end{bmatrix}
-
\begin{bmatrix}
  \mu_{00} & \mu_{01} & \cdots & \mu_{0n} \\
  \mu_{10} & \mu_{11} & \cdots & \mu_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  \mu_{m0} & \mu_{m1} & \cdots & \mu_{mn} \\
\end{bmatrix} 
\right)
\right|
= |\mathbf{a} - \mathbf{b}| \tag{2}
$$

In [ ]:
# 3.
assert np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_one_loop((samples_test - mu) / std, (samples_train - mu) / std) * std,
)

$$
|\tilde{\mathbf{a}} - \tilde{\mathbf{b}}|
=
\left|
\left(
\begin{bmatrix}
  a_{00} & a_{01} & \cdots & a_{0n} \\
  a_{10} & a_{11} & \cdots & a_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  a_{m0} & a_{m1} & \cdots & a_{mn} \\
\end{bmatrix} 
- \mu
\right)/\sigma
-
\left(
\begin{bmatrix}
  b_{00} & b_{01} & \cdots & b_{0n} \\
  b_{10} & b_{11} & \cdots & b_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  b_{m0} & b_{m1} & \cdots & b_{mn} \\
\end{bmatrix}
- \mu
\right)/\sigma
\right|
= \left|\frac{\mathbf{a} - \mathbf{b}}{\sigma}\right| \tag{3}
$$

In [ ]:
# 4.
assert not np.allclose(
    np.argsort(compute_image_l1_distance_one_loop(samples_test, samples_train)),
    np.argsort(compute_image_l1_distance_one_loop((samples_test - mu_ij) / std_ij, (samples_train - mu_ij) / std_ij)),
)

$$
|\tilde{\mathbf{a}} - \tilde{\mathbf{b}}|
=
\left|
\frac{\left(
\begin{bmatrix}
  a_{00} & a_{01} & \cdots & a_{0n} \\
  a_{10} & a_{11} & \cdots & a_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  a_{m0} & a_{m1} & \cdots & a_{mn} \\
\end{bmatrix} 
-
\begin{bmatrix}
  \mu_{00} & \mu_{01} & \cdots & \mu_{0n} \\
  \mu_{10} & \mu_{11} & \cdots & \mu_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  \mu_{m0} & \mu_{m1} & \cdots & \mu_{mn} \\
\end{bmatrix} 
\right)}
{
\begin{bmatrix}
  \sigma_{00} & \sigma_{01} & \cdots & \sigma_{0n} \\
  \sigma_{10} & \sigma_{11} & \cdots & \sigma_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  \sigma_{m0} & \sigma_{m1} & \cdots & \sigma_{mn} \\
\end{bmatrix}
}
-
\frac{\left(
\begin{bmatrix}
  b_{00} & b_{01} & \cdots & b_{0n} \\
  b_{10} & b_{11} & \cdots & b_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  b_{m0} & b_{m1} & \cdots & b_{mn} \\
\end{bmatrix}
-
\begin{bmatrix}
  \mu_{00} & \mu_{01} & \cdots & \mu_{0n} \\
  \mu_{10} & \mu_{11} & \cdots & \mu_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  \mu_{m0} & \mu_{m1} & \cdots & \mu_{mn} \\
\end{bmatrix} 
\right)}
{
\begin{bmatrix}
  \sigma_{00} & \sigma_{01} & \cdots & \sigma_{0n} \\
  \sigma_{10} & \sigma_{11} & \cdots & \sigma_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  \sigma_{m0} & \sigma_{m1} & \cdots & \sigma_{mn} \\
\end{bmatrix}
}
\right|
=
\left|
\begin{bmatrix}
  (a_{00}-b_{00})/\sigma_{00} & (a_{01}-b_{01})/\sigma_{01} & \cdots & (a_{0n}-b_{0n})/\sigma_{0n} \\
  (a_{10}-b_{10})/\sigma_{10} & (a_{11}-b_{11})/\sigma_{11} & \cdots & (a_{1n}-b_{1n})/\sigma_{1n} \\
  \vdots & \vdots & \ddots & \vdots \\
  (a_{m0}-b_{m0})/\sigma_{m0} & (a_{m1}-b_{m1})/\sigma_{m1} & \cdots & (a_{mn}-b_{mn})/\sigma_{mn} \\
\end{bmatrix} 
\right|
\neq |\mathbf{a} - \mathbf{b}| \tag{4}
$$

$d_{ij} =\sum{|\mathbf{a^{[i]}} - \mathbf{b^{[i]}}|}$, Let

$$
\sum{
\left|
\begin{bmatrix}
  0 & 0 & 0 \\
  0 & 0 & 0 \\
  0 & 0 & 9 \\
\end{bmatrix}
\right|
}=9
\gt
\sum{
\left|
\begin{bmatrix}
  1 & 1 & 1 \\
  1 & 1 & 1 \\
  1 & 1 & 0 \\
\end{bmatrix}
\right|
} =8
\Longrightarrow
\sum{
\left|
\frac{
\begin{bmatrix}
  0 & 0 & 0 \\
  0 & 0 & 0 \\
  0 & 0 & 9 \\
\end{bmatrix}
-\begin{bmatrix}
  0 & 0 & 0 \\
  0 & 0 & 0 \\
  0 & 0 & 0 \\
\end{bmatrix}
}
{\begin{bmatrix}
  1 & 1 & 1 \\
  1 & 1 & 1 \\
  1 & 1 & 9 \\
\end{bmatrix}}
\right|
}=1
\lt
\sum{
\left|
\frac{
\begin{bmatrix}
  1 & 1 & 1 \\
  1 & 1 & 1 \\
  1 & 1 & 0 \\
\end{bmatrix}
-\begin{bmatrix}
  0 & 0 & 0 \\
  0 & 0 & 0 \\
  0 & 0 & 0 \\
\end{bmatrix}
}
{\begin{bmatrix}
  1 & 1 & 1 \\
  1 & 1 & 1 \\
  1 & 1 & 9 \\
\end{bmatrix}}
\right|
}=8
$$

In [ ]:
# 5.
assert np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_one_loop(np.rot90(samples_test, axes=(1, 2)), np.rot90(samples_train, axes=(1, 2))),
)

assert np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_one_loop(
        np.rot90(samples_test, k=2, axes=(1, 2)), np.rot90(samples_train, k=2, axes=(1, 2))
    ),
)

assert not np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_one_loop(
        rotate(samples_test, angle=45, axes=(1, 2)), rotate(samples_train, angle=45, axes=(1, 2))
    ),
)

assert not np.allclose(
    compute_image_l1_distance_one_loop(samples_test, samples_train),
    compute_image_l1_distance_one_loop(
        rotate(samples_test, angle=30, axes=(1, 2)), rotate(samples_train, angle=30, axes=(1, 2))
    ),
)